[View in Colaboratory](https://colab.research.google.com/github/thispassing/colaboratory/blob/master/oct_14_feature_sets_study.ipynb)

In [2]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))

print(california_housing_dataframe)


       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
3930      -118.0      34.0                34.0       1381.0           265.0   
1121      -117.1      33.1                45.0       1032.0           235.0   
7419      -118.3      34.0                30.0       2661.0           765.0   
12106     -121.4      37.7                52.0        876.0           170.0   
15227     -122.3      37.8                52.0        210.0            56.0   
...          ...       ...                 ...          ...             ...   
212       -116.4      33.5                26.0        186.0            48.0   
16463     -122.6      38.2                18.0       2933.0           481.0   
10085     -119.8      34.4                22.0       2845.0           500.0   
8451      -118.5      34.2                12.0       3831.0          1083.0   
8494      -118.5      34.2                28.0       4054.0           712.0   

       population  households  median_income  media

In [0]:
def preprocess_features(california_housing_dataframe):
  selected_features = california_housing_dataframe[
      ["latitude",
      "longitude",
      "housing_median_age",
      "total_rooms",
      "total_bedrooms",
      "population",
      "households",
      "median_income"]]
  
  processed_features = selected_features.copy()
  processed_features["rooms_per_person"] = (
      california_housing_dataframe["total_rooms"] /
      california_housing_dataframe["population"])
  return processed_features

def preprocess_targets(california_housing_dataframe):
  output_targets = pd.DataFrame()
  output_targets["median_house_value"] = (
      california_housing_dataframe["median_house_value"] / 1000.0)
  return output_targets

In [0]:
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_targets = preprocess_targets(california_housing_dataframe.head(12000))

validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))

# print("Training examples summary")
# display.display(training_examples.describe())

# print("Training targets summary")
# display.display(training_targets.describe())

# print("Validation examples summary")
# display.display(validation_examples.describe())

# print("Validation targets summary")
# display.display(validation_targets.describe())



In [5]:
correlation_dataframe = training_examples.copy()
correlation_dataframe["target"] = training_targets["median_house_value"]
correlation_dataframe.corr()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person,target
latitude,1.0,-0.9,0.0,-0.0,-0.1,-0.1,-0.1,-0.1,0.1,-0.1
longitude,-0.9,1.0,-0.1,0.0,0.1,0.1,0.1,-0.0,-0.1,-0.0
housing_median_age,0.0,-0.1,1.0,-0.4,-0.3,-0.3,-0.3,-0.1,-0.1,0.1
total_rooms,-0.0,0.0,-0.4,1.0,0.9,0.9,0.9,0.2,0.1,0.1
total_bedrooms,-0.1,0.1,-0.3,0.9,1.0,0.9,1.0,-0.0,0.1,0.1
population,-0.1,0.1,-0.3,0.9,0.9,1.0,0.9,0.0,-0.1,-0.0
households,-0.1,0.1,-0.3,0.9,1.0,0.9,1.0,0.0,-0.0,0.1
median_income,-0.1,-0.0,-0.1,0.2,-0.0,0.0,0.0,1.0,0.2,0.7
rooms_per_person,0.1,-0.1,-0.1,0.1,0.1,-0.1,-0.0,0.2,1.0,0.2
target,-0.1,-0.0,0.1,0.1,0.1,-0.0,0.1,0.7,0.2,1.0


In [0]:
def construct_feature_columns(input_features):
 return set([tf.feature_column.numeric_column(my_feature)
            for feature in input_features])

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
  features = {key:np.array(value) for key,value in dict(features).items()}
  
  ds = Dataset.from_tensor_slices((features,targets))
  ds = ds.batch(batch_size).repeat(num_epochs)
  
  if shuffle:
    ds = ds.shuffle(10000)
   
  features, labels = ds.make_one_shot_iterator().get_next()
  return features, labels
  
  

In [0]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    trainging_targets,
    validation_examples,
    validation_targets):
  
  periods = 10
  steps_per_period = steps/periods
  
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns = construct_feature_columns(training_examples),
      optimizer = my_optimizer)
  
  training_input_fn = lambda: my_input_fn(training_examples,
                                         training_targets["median_house_value"],
                                         batch_size=batch_size)
  
  predict_training_input_fn = lambda: my_input_fn(training_examples,
                                                 training_targets["median_house_value"],
                                                 num_epochs = 1,
                                                 shuffle=False)
  
  predict_validation_input_fn = lambda: my_input_fn(validation_examples,
                                                   validation_targets["median_house_value"],
                                                   num_epochs = 1,
                                                   shuffle=False)

  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  for period in range(0,periods):
    linear_regressor.train(
        input_fn=training_input_fn,
        steps = steps_per_period)
  
    training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
  
    validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item]['predictions'][0] for item in validation_predictions)
  
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
  
    print(" period %02d : %0.2f" % period, training_root_mean_squared_error)
  
    training.rmse.append(training_root_mean_squared_error)
    validation.rmse.append(validation_root_mean_squared_error)
  
  print("Model training finished.")

  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  return linear_regressor
  